### Baryonic Tully-Fisher relation

Construct Tully-Fisher (Lr-Vcirc) or baryonic Tully-Fisher $M_*+M_{\rm HI}$-Vcirc using ALFALFA a40 catalog and SDSS DR7 galaxies matched to it. We will use <a href="http://www.physics.upenn.edu/~ameert/SDSS_PhotDec/download/">UPenn catalogs</a> with their model photometry fits to the SDSS DR7 main galaxy sample (<a href="http://adsabs.harvard.edu/abs/2015MNRAS.446.3943M">Meert et al. 2015</a>). Place all fits files in the data/Meert2015_v2 subdirectory within the directory containing this notebook and other python code files. We will also use probabilities of morphological classes from Huertas-Company et al. (2011) that are supplied as part of this catalog.

We will also need <a href="https://bitbucket.org/bdiemer/colossus/overview"><tt>colossus</tt></a> - Benedikt Diemer's  python package containing useful routines for computing various cosmological quantities (distances, variances, power spectra, halo profiles, etc.)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate, stats

from read_data import read_alfalfa
from read_data import read_alfalfa_sdss_crosslist
from read_data import alfalfa_sdss_crossmatch

# read ALFALFA data and SDSS cross-listing table
aalist   = read_alfalfa('data//a40.datafile1.txt');
sdsslist = read_alfalfa_sdss_crosslist('data//a40.datafile3.txt')
# cross-match the catalogs and perform basic cuts
aatf, sdsstf = alfalfa_sdss_crossmatch(aalist, sdsslist)

# read in the relevant data from Meert et al. (2015) catalogs
from read_data import read_meert_catalog
sdata, mdata, mnpdata, phot_r, mdatag, mnpdatag, morph = read_meert_catalog(phot_type = 3)

#
# match ALFALFA and SDSS catalogs using photoObjID
#
imatch = np.in1d(sdsstf['PhotoObjID'],sdata['objid'])
imatch2 = np.in1d(sdata['objid'],sdsstf['PhotoObjID'])
aatfmatch = aatf[imatch]; sdssmatch = sdsstf[imatch]
datamatch = sdata[imatch2]; photmatch = phot_r[imatch2]; 
mdatamatch = mdata[imatch2]; mdatagmatch = mdatag[imatch2]; 
mnpdmatch = mnpdata[imatch2]; mnpdgmatch = mnpdatag[imatch2]; 
morphmatch = morph[imatch2]

# import Benedikt Diemer's colossus package
from colossus.cosmology import cosmology

# set cosmology to the best values from 9-year WMAP data
cosmo = cosmology.setCosmology('WMAP9')

# extract relevand data
# redshift and log10(HI mass)
z = sdssmatch['zsdss']; lMHI = aatfmatch['logMsun']
# compute luminosity distance in Mpc
d_Lm = cosmo.luminosityDistance(z)/cosmo.h

extm = mnpdmatch['extinction']; kcorr = mnpdmatch['kcorr']
# abs. magnitude from the Meert et al. photometry using fit specified by phot_type above
# corrected for extinction, evolution, and k-correction
Mr = mdatamatch['m_tot'] - 5.0*np.log10(d_Lm/1e-5) - extm + 1.3*z - kcorr;
gr = mdatagmatch['m_tot'] - mdatamatch['m_tot']  - mnpdgmatch['extinction'] + mnpdmatch['extinction']
ur = sdssmatch['uminusr']
ba = mdatamatch['ba_tot']
cos2 = (np.power(ba,2)-0.13*0.13)/(1.0-0.13*0.13)
sini = np.sqrt(1.0-cos2)
# convert line width into rotation velocity (ostensibly)
s = 0.5*aatfmatch['W50']/sini

#
# assigned probability for galaxy to be of particular type from Huertas-Company et al. 2011 classification
#
pSab = morphmatch['probaSab']; pScd = morphmatch['probaScd']; pS0  = morphmatch['probaS0']

# maximize disks, get rid of likely S0's
pdisk = 0.7
indplot = (((pSab > pdisk) | (pScd > pdisk)) & (pS0<0.3))
Mr = Mr[indplot]; s = s[indplot]; lMHI = lMHI[indplot]
ur = ur[indplot]; gr = gr[indplot]
Lr = np.power(10.,-0.4*(Mr-4.68))
addHI = False
if addHI:
    # get the  r-band M/L ratio from color using 
    # linear mapping from (u-r) or (g-r) color from Table 7 of 
    # Bell et al. 2003, ApJS 149, 289 derived from SPS 
    #lMLr = -0.223 + 0.299*ur
    lMLr = -0.306 + 1.097*gr
    # convert Lr to stellar mass, add HI 
    Lr = np.clip(lMLr*Lr,1.e3,1.e15) + np.power(10.,lMHI)

# now plot the relation
#
fig = plt.figure(figsize=(3, 3))
plt.rc('font',size=11)

if addHI:
    plt.xlabel(r'$M_*+M_{\mathrm{HI}}\ (M_{\odot})$')
else:
    plt.xlabel(r'$L_r$')
plt.ylabel(r'$V_{\rm rot}\ (\mathrm{km/s})$')
plt.ylim(10.,700.); 
plt.xlim(1.e7, 3.e11);
plt.yscale('log'); plt.xscale('log')

from scipy.stats import binned_statistic
cmed, ibedges, NMbins = binned_statistic(np.log10(Lr),s,statistic='median',bins=20)

ibins = 0.5*(ibedges[1:] + ibedges[:-1])

plt.scatter(Lr, s, marker='.', color='blue', s=0.75, alpha=0.5)
plt.plot(10.**ibins,cmed,c='m',lw=4.0)

def perc84(x):
    return np.percentile(x,84.0)
def perc16(x):
    return np.percentile(x,16.0)
    
s16, ibedges, NMbins = binned_statistic(np.log10(Lr),s,statistic=perc16,bins=20)
s84, ibedges, NMbins = binned_statistic(np.log10(Lr),s,statistic=perc84,bins=20)
plt.plot(10.**ibins, s16,'--',c='m',lw=3.0)
plt.plot(10.**ibins, s84,'--',c='m',lw=3.0)

# plot the L~Vmax^a power law appropriate for a given relation
x = np.linspace(6., 12., 100)
if not addHI:
    y = 85.*10**((x-9.)/4.3); plabel = '$L\propto V^{4.3}$'
else:
    y = 65.*10**((x-9.)/3.3); plabel = '$M_*+M_{\mathrm{HI}}\propto V^{3.3}$'

plt.plot(10.**x,y,'--', c='r', lw=3.0, label=plabel)

plt.legend(frameon=False, fontsize=10, loc='upper left')
plt.show()


670722 galaxies in Meert et al. sample initially
